In [1]:
import warnings

import pandas as pd
import seaborn as sns

from utils.parsers import Parser

warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv("data/habr/raw_texts.csv")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166320 entries, 0 to 166319
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  166320 non-null  int64 
 1   id          166320 non-null  int64 
 2   title       166320 non-null  object
 3   text        165728 non-null  object
 4   tags        166320 non-null  object
dtypes: int64(2), object(3)
memory usage: 6.3+ MB


In [11]:
def custom_str_to_lst(string):
    return string[1:-1].replace("'", "").split(", ")

In [12]:
df['tags'] = df.tags.apply(custom_str_to_lst)

In [35]:
(df['tags']
 .explode()
 .apply(lambda x: x.lower())
 .value_counts()
 .index[:200]
 .tolist())

['google',
 'android',
 'javascript',
 'microsoft',
 'linux',
 'php',
 'apple',
 'java',
 'python',
 'программирование',
 'стартапы',
 'разработка',
 'ios',
 'стартап',
 'социальные сети',
 'iphone',
 '.net',
 'яндекс',
 'windows',
 'css',
 'игры',
 'дизайн',
 'c++',
 'open source',
 'хабрахабр',
 'безопасность',
 'интернет',
 '',
 'facebook',
 'реклама',
 'информационная безопасность',
 'видео',
 'юмор',
 'firefox',
 'конференция',
 'c#',
 'html',
 'html5',
 'ubuntu',
 'opera',
 'статистика',
 'юзабилити',
 'тестирование',
 'обзор',
 'маркетинг',
 'управление проектами',
 'перевод',
 'ruby',
 'jquery',
 'работа',
 'хостинг',
 'twitter',
 'бизнес',
 'nokia',
 'web-разработка',
 'конкурс',
 'api',
 'браузеры',
 'mail.ru',
 'mysql',
 'новости',
 'образование',
 'samsung',
 'intel',
 'c',
 'поиск',
 'инвестиции',
 'обучение',
 'node.js',
 'youtube',
 'оптимизация',
 'веб 2.0',
 'вконтакте',
 'flash',
 'ссылки',
 'windows phone',
 'дайджест',
 'веб-разработка',
 'алгоритмы',
 'музыка',
 'к

In [36]:
tags_to_save = set([
 'javascript',
 'linux',
 'php',
 'java',
 'python',
 'программирование',
 'разработка',
 '.net',
 'c++',
 'c#',
 'ubuntu',
 'ruby',
 'web-разработка',
 'api',
 'mysql',
 'c',
 'node.js',
 'веб-разработка',
 'алгоритмы',
 'виртуализация',
 'qt',
 'интерфейсы',
 'sql','машинное обучение', 'ml', 'dl', 'machine learning','docker'])

In [37]:
mask = df.tags.apply(lambda x: bool(list(set(x) & tags_to_save)))

In [45]:
res = df[mask]

In [46]:
res["text"] = res.text.apply(str)
res["text_len"] = res.text.apply(len)
res.text_len.mean()

C:\Users\ilove\AppData\Local\Temp\ipykernel_14728\3953190809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res["text"] = res.text.apply(str)
C:\Users\ilove\AppData\Local\Temp\ipykernel_14728\3953190809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res["text_len"] = res.text.apply(len)


8207.561475581639

In [47]:
res.reset_index(drop=True, inplace=True)

In [48]:
res.to_csv("data/habr/clean_habr.csv")

In [72]:
parser = Parser()

sents = []
for text in tqdm(res.text):
    sents += parser.text_to_sents(text)

100%|███████████████████████████████████████████████████████████████████████████| 23855/23855 [02:01<00:00, 196.24it/s]


In [78]:
df = pd.DataFrame({"text": sents})
df.to_csv("data/sents/habr.csv", encoding='utf-8')